# PYTORCH CNN Classifier

In [1]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [2]:
# Parameters
DATASET = "demo_coding_vs_intergenomic_seqs"
TOKENIZER = "kmer"
KMER = 3


In [3]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

demo_coding_vs_intergenomic_seqs 0 kmer 3


## Config

In [4]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [5]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 66
{'ACG': 64, 'TCG': 63, 'GCG': 62, 'CCG': 61, 'CGC': 59, 'CTA': 58, 'ATT': 57, 'AGG': 55, 'CTT': 56, 'CGG': 54, 'CTC': 53, 'ACC': 49, 'AGC': 48, 'GTA': 4, 'ATA': 14, 'CGT': 45, 'GGC': 44, '<pad>': 65, 'TCA': 7, 'CAG': 43, 'GTT': 38, 'CCA': 37, 'CAA': 34, 'AAG': 32, 'CAT': 26, 'GCT': 25, 'CTG': 23, 'TCC': 39, 'AAC': 22, 'TTC': 21, 'ATG': 15, 'GAA': 41, 'CCC': 47, 'GAC': 20, 'AGT': 40, 'TTA': 33, 'TGG': 18, 'ATC': 6, 'TAT': 8, 'TAC': 19, 'ACA': 16, 'AAA': 3, 'TTG': 29, 'GGA': 1, 'AAT': 13, 'TGT': 11, 'CGA': 60, 'GTC': 52, 'GCC': 30, 'TTT': 10, 'GGG': 50, 'GAT': 31, 'TGC': 28, 'AGA': 36, 'CAC': 27, '<unk>': 0, 'CCT': 5, 'GTG': 46, 'TAA': 51, 'TCT': 42, 'ACT': 17, 'GAG': 12, 'TAG': 2, 'TGA': 35, 'GCA': 24, 'GGT': 9}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device


max_tok_len  66


In [9]:
tokenizer(train_dset[1][0])

['CCA',
 'TCC',
 'GAG',
 'TGT',
 'CCT',
 'AGT',
 'GAA',
 'TCT',
 'GCA',
 'GCC',
 'CAG',
 'ATG',
 'GGC',
 'TCA',
 'CGT',
 'TTC',
 'CCT',
 'GTG',
 'TAC',
 'CCC',
 'ACA',
 'TGC',
 'CCA',
 'CTG',
 'AGC',
 'CCC',
 'GAA',
 'GTG',
 'GTT',
 'GCC',
 'CAG',
 'ACC',
 'CCT',
 'TAC',
 'CCA',
 'GAC',
 'CCC',
 'GGG',
 'CTG',
 'AAA',
 'GAA',
 'CCT',
 'TGT',
 'GTG',
 'GTT',
 'TAA',
 'TCT',
 'CAG',
 'AGC',
 'AGA',
 'GTC',
 'CTC',
 'TGA',
 'GCT',
 'GGA',
 'GAT',
 'GTT',
 'TGA',
 'TTG',
 'AGA',
 'GAC',
 'TGC',
 'CGG',
 'AGG',
 'GGC',
 'CTC']

## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 81.2%, Avg loss: 0.584524 

Epoch 1


Train metrics: 
 Accuracy: 82.9%, Avg loss: 0.581704 

Epoch 2


Train metrics: 
 Accuracy: 84.2%, Avg loss: 0.573939 

Epoch 3


Train metrics: 
 Accuracy: 84.6%, Avg loss: 0.573182 

Epoch 4


Train metrics: 
 Accuracy: 85.1%, Avg loss: 0.569501 



## Testing

In [12]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  12500 ; tp  9551.149959802628 ; fp  1296.3022161659421
recall  0.7640919967842102 ; precision  0.8804970793936507
num_batches 782
correct 20771
size 25000
Test metrics: 
 Accuracy: 0.830840, F1 score: 0.818175, Avg loss: 0.578670 



(0.83084, 0.8181749244256797)